In [7]:
import pandas as pd
from pulp import *
import requests

In [8]:
def get_data(team_id, gw, team_override=False, team=[]):
    # get FPL api data
    r = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
    fpl_data = r.json()
    
    # get FPL player and team data
    element_data = pd.DataFrame(fpl_data['elements'])
    team_data = pd.DataFrame(fpl_data['teams'])
    type_data = pd.DataFrame(fpl_data['element_types']).set_index(['id'])
    elements_team = pd.merge(element_data, team_data, left_on='team', right_on='id')
                        
    # get fplreview data and create final merged table
    review_data = pd.read_csv('../data/fplreview_gw6.csv')
    review_data['review_id'] = review_data.index+1
    merged_data = pd.merge(elements_team, review_data, left_on='id_x', right_on='review_id')
    merged_data.set_index(['id_x'], inplace=True)
    next_gw = int(review_data.keys()[5].split('_')[0])
    
    # get my team data
    if not team_override:
        r = requests.get(f'https://fantasy.premierleague.com/api/entry/{team_id}/event/{gw}/picks/')
        picks_data = r.json()
        initial_squad = [i['element'] for i in picks_data['picks']]
        r = requests.get(f'https://fantasy.premierleague.com/api/entry/{team_id}/')
        general_data = r.json()
        itb = general_data['last_deadline_bank'] / 10
    else:
        initial_squad = team
        if len(team) == 0:
            itb = 100
        else:
            itb = 0
    
    return {
        'merged_data': merged_data,
        'elements_team': elements_team,
        'team_data' : team_data,
        'type_data' : type_data,
        'next_gw' : next_gw,
        'initial_squad' : initial_squad,
        'itb': itb
    }

In [9]:
def multi_period_solver(team_id, gw, ft, horizon, objective='regular', decay_base=0.84, team_override=False, team=[]):
    
    # load data
    data = get_data(team_id, gw-1, team_override=team_override, team=team)
    merged_data = data['merged_data']
    team_data = data['team_data']
    type_data = data['type_data']
    #next_gw = data['next_gw']
    next_gw = gw
    initial_squad = data['initial_squad']
    itb = data['itb']
    problem_name = f'mp_b{itb}_h{horizon}_o{objective}_d{decay_base}'
    
    # prepare data
    players = merged_data.index.to_list()
    element_types = type_data.index.to_list()
    teams = team_data['name'].to_list()
    gameweeks = list(range(next_gw, next_gw + horizon))
    all_gws = [next_gw-1] + gameweeks
    
    # initialize solver
    prob = LpProblem(problem_name, LpMaximize)
    
    # set variables
    squad = LpVariable.dicts('squad', (players, all_gws), 0, 1, cat='Binary')
    lineup = LpVariable.dicts('lineup', (players, gameweeks), 0, 1, cat='Binary')
    captain = LpVariable.dicts('captain', (players, gameweeks), 0, 1, cat='Binary')
    vicecap = LpVariable.dicts('vicecap', (players, gameweeks), 0, 1, cat='Binary')
    transfer_in = LpVariable.dicts('transfer_in', (players, gameweeks), 0, 1, cat='Binary')
    transfer_out = LpVariable.dicts('transfer_out', (players, gameweeks), 0, 1, cat='Binary')
    in_the_bank = LpVariable.dicts('itb', all_gws, lowBound=0, cat="Continuous")
    free_transfers = LpVariable.dicts('ft', all_gws, lowBound=1, upBound=2, cat="Integer")
    penalized_transfers = LpVariable.dicts('pt', gameweeks, lowBound=0, cat="Integer")
    aux = LpVariable.dicts('aux', gameweeks, 0, 1, cat='Binary')
    
    # dictionaries
    lineup_type_count = {(t, w): lpSum(lineup[p][w] for p in players if merged_data.loc[p, 'element_type'] == t) for t in element_types for w in gameweeks}
    squad_type_count = {(t, w): lpSum(squad[p][w] for p in players if merged_data.loc[p, 'element_type'] == t) for t in element_types for w in gameweeks}
    player_price = (merged_data['now_cost'] / 10).to_dict()
    sold_amount = {w: lpSum(player_price[p] * transfer_out[p][w] for p in players) for w in gameweeks}
    bought_amount = {w: lpSum(player_price[p] * transfer_in[p][w] for p in players) for w in gameweeks}
    points_player_week = {(p, w): merged_data.loc[p, f'{w}_Pts'] for p in players for w in gameweeks}
    squad_count = {w: lpSum(squad[p][w] for p in players) for w in gameweeks}
    lineup_count = {w: lpSum(lineup[p][w] for p in players) for w in gameweeks}
    captain_count = {w: lpSum(captain[p][w] for p in players) for w in gameweeks}
    vicecap_count = {w: lpSum(vicecap[p][w] for p in players) for w in gameweeks}
    number_of_transfers = {w: lpSum(transfer_out[p][w] for p in players) for w in gameweeks}
    number_of_transfers[next_gw-1] = 1
    transfer_diff = {w: number_of_transfers[w] - free_transfers[w] for w in gameweeks}
    
    # initial constraints
    if team_override and len(team) == 0:
        for p in players:
            prob += squad[p][next_gw-1] == 0
            
    else:
        for p in initial_squad:
            prob += squad[p][next_gw-1] == 1 
        for p in players:
            if p not in initial_squad:
                prob += squad[p][next_gw-1] == 0
    prob += in_the_bank[next_gw-1] == itb
    prob += free_transfers[next_gw-1] == ft
    
    # force exclude
    for p in players:
        for w in gameweeks:
            if p in [154]: #,268, 212]:
                prob += squad[p][w] == 0
    
    # set constraints
    for w in gameweeks:
        prob += squad_count[w] == 15
        prob += lineup_count[w] == 11 
        prob += captain_count[w] == 1
        prob += vicecap_count[w] == 1
        
        # free transfer constraints
        prob += free_transfers[w] == aux[w] + 1
        prob += free_transfers[w-1] - number_of_transfers[w] <= 2 * aux[w]
        prob += free_transfers[w-1] - number_of_transfers[w] >= aux[w] + (-14)*(1 - aux[w])
        prob += penalized_transfers[w] >= transfer_diff[w]
        prob += in_the_bank[w] == in_the_bank[w-1] + sold_amount[w] - bought_amount[w]
        
        for p in players:
            prob += squad[p][w] >= lineup[p][w]
            prob += lineup[p][w] >= captain[p][w]
            prob += lineup[p][w] >= vicecap[p][w]
            prob += captain[p][w] + vicecap[p][w] <= 1
            
            # transfer constraints
            prob += squad[p][w] == squad[p][w-1] + transfer_in[p][w] - transfer_out[p][w]
            
        for t in element_types:
            prob += lineup_type_count[t, w] <= type_data.loc[t, 'squad_max_play']
            prob += lineup_type_count[t, w] >= type_data.loc[t, 'squad_min_play']
            prob += squad_type_count[t, w] == type_data.loc[t, 'squad_select']
        for t in teams:
            prob += lpSum(squad[p][w] for p in players if merged_data.loc[p, 'Team'] == t) <= 3
    
    # set objective
    gw_xp = {w: lpSum(points_player_week[p, w] * (lineup[p][w] + captain[p][w] + 0.1* vicecap[p][w]) for p in players) for w in gameweeks}
    gw_total = {w: gw_xp[w] - 4 * penalized_transfers[w] for w in gameweeks}
    if objective == 'regular':
        total_xp = lpSum(gw_total[w] for w in gameweeks)
        prob += total_xp
    else:
        decay_objective = lpSum(gw_total[w] * pow(decay_base, w-next_gw) for w in gameweeks)
        prob += decay_objective
    
    # solve
    prob.writeLP(f'../lp_files/{problem_name}.lp');
    prob.solve(PULP_CBC_CMD(msg=0));
    picks = []
    for w in gameweeks:
        for p in players:
            is_cap = 0
            is_vice = 0
            is_lineup = 0
            is_transfer_in = 0
            is_transfer_out = 0
            if squad[p][w].varValue + transfer_out[p][w].varValue == 1:
                if captain[p][w].varValue == 1:
                    is_cap = 1
                if vicecap[p][w].varValue == 1:
                    is_vice = 1
                if lineup[p][w].varValue == 1:
                    is_lineup = 1
                if transfer_in[p][w].varValue == 1:
                    is_transfer_in = 1
                if transfer_out[p][w].varValue == 1:
                    is_transfer_out = 1
                picks.append([
                    w,
                    type_data.loc[merged_data.loc[p, 'element_type'], 'singular_name_short'],
                    merged_data.loc[p, 'element_type'],
                    merged_data.loc[p, 'web_name'],
                    merged_data.loc[p, 'name'],
                    player_price[p],
                    points_player_week[p, w], 
                    is_lineup, 
                    is_cap, 
                    is_vice,
                    is_transfer_in,
                    is_transfer_out
                ])
    picks_df = pd.DataFrame(picks, columns=[
        'week',
        'pos', 
        'element_type', 
        'name', 
        'team', 
        'price', 
        'xPts', 
        'is_lineup', 
        'is_cap', 
        'is_vice',
        'is_transfer_in',
        'is_transfer_out'
    ]).sort_values(by=[
        'week', 
        'is_lineup', 
        'element_type', 
        'xPts'
    ], ascending=[
        True,
        False,
        True,
        True
    ])
    total_points = value(lpSum((lineup[p][w] + captain[p][w]) * points_player_week[p, w] for p in players for w in gameweeks))
    summary_of_actions = ""
    for w in gameweeks:
        summary_of_actions += f"** GW {w}:\n"
        summary_of_actions += f"ITB - {in_the_bank[w].varValue}, FT - {free_transfers[w].varValue}, PT - {penalized_transfers[w].varValue} , xPts - {round(value(lpSum((lineup[p][w] + captain[p][w]) * points_player_week[p, w] for p in players)), 2)}\n"
        for p in players:
            if transfer_in[p][w].varValue == 1:
                summary_of_actions += f"Buy {p} - {merged_data['web_name'][p]}\n"
            if transfer_out[p][w].varValue == 1:
                summary_of_actions += f"Sell {p} - {merged_data['web_name'][p]}\n"

    return {
        'picks': picks_df, 
        'total_xPts': total_points, 
        'summary': summary_of_actions
    }



In [14]:
sol = multi_period_solver(
    team_id=167302, 
    gw=6, 
    ft=1, 
    horizon=8, 
    objective='decay', 
    #team_override=True, 
    #team=[69, 237, 142, 275, 437, 233, 240, 196, 430, 189, 579, 54, 491, 354, 163] #-- my WC
    #team=[] # optimization WC
)
print(sol['summary'])

** GW 6:
ITB - 0.5, FT - 1.0, PT - 0.0 , xPts - 56.62
Sell 189 - Bamford
Buy 413 - Antonio
** GW 7:
ITB - 1.0, FT - 1.0, PT - 0.0 , xPts - 56.03
Buy 17 - Pépé
Sell 240 - Jota
** GW 8:
ITB - 0.3, FT - 1.0, PT - 0.0 , xPts - 56.59
Buy 127 - Rüdiger
Sell 437 - Semedo
** GW 9:
ITB - 1.4, FT - 1.0, PT - 0.0 , xPts - 54.47
Buy 529 - Lukaku
Sell 579 - Ronaldo
** GW 10:
ITB - 0.4, FT - 1.0, PT - 0.0 , xPts - 55.61
Buy 96 - Mbeumo
Sell 354 - Sissoko
** GW 11:
ITB - 2.0, FT - 1.0, PT - 0.0 , xPts - 56.82
Buy 78 - Toney
Sell 413 - Antonio
** GW 12:
ITB - 1.6, FT - 1.0, PT - 0.0 , xPts - 52.82
Sell 142 - James
Buy 256 - Cancelo
** GW 13:
ITB - 0.2, FT - 1.0, PT - 0.0 , xPts - 56.82
Sell 127 - Rüdiger
Buy 234 - Robertson



In [17]:
# display GW5 team
print(sol['picks'].loc[sol['picks']['week'] == 5])

    week  pos  element_type              name       team  xPts  price  \
0      5  GKP             1           Sánchez   Brighton   4.5  3.719   
8      5  DEF             2             Matip  Liverpool   5.0  4.732   
1      5  DEF             2             James    Chelsea   5.6  4.135   
11     5  DEF             2           Cancelo   Man City   6.0  5.160   
10     5  DEF             2  Alexander-Arnold  Liverpool   7.5  6.058   
7      5  MID             3          Raphinha      Leeds   6.5  4.811   
12     5  MID             3            Torres   Man City   7.2  6.229   
9      5  MID             3             Salah  Liverpool  12.5  7.572   
14     5  FWD             4           Jiménez     Wolves   7.4  4.547   
6      5  FWD             4           Bamford      Leeds   7.9  4.895   
13     5  FWD             4           Ronaldo    Man Utd  12.6  5.555   
3      5  GKP             1           Begović    Everton   4.0  0.081   
5      5  DEF             2            Ayling      